In [1]:
import numpy as np
import pandas as pd


def augmente_data(raw_df, x_cols, y_cols, n_augmented=0,
                  y_std_cols=None, x_std=0, y_std=0,
                  id_col='sID', da_col='DA#', random_state=None):
    if y_std_cols is not None:
        y_std_df = raw_df[std_cols]
    else:
        y_std_df = np.ones(raw_df[y_cols].values.shape) * y_std

    x_std_df = np.ones(raw_df[x_cols].values.shape) * x_std

    raw_df = raw_df.copy()
    raw_df[da_col] = 0

    noisy_dfs = [raw_df, ]
    rng = np.random.default_rng(random_state)

    for i in range(n_augmented):
        noisy_x = pd.DataFrame(rng.normal(raw_df[x_cols], x_std_df), columns=x_cols)
        noisy_y = pd.DataFrame(rng.normal(raw_df[y_cols], y_std_df), columns=y_cols)
        noisy_xy = noisy_x.join(noisy_y)
        noisy_sid = raw_df[[id_col]].copy()
        noisy_sid[da_col] = i + 1
        noisy_df = noisy_sid.join(noisy_xy).reset_index(drop=True)
        noisy_dfs.append(noisy_df)
    final_df = pd.concat(noisy_dfs, ignore_index=True).sort_values(by=[id_col, da_col]).reset_index(drop=True)
    return final_df[[id_col, da_col] + x_cols + y_cols]

In [2]:
df_train = pd.read_csv('active_learning_training_data.csv')
X_COLS = ['CNT', 'MXN', 'GNP', 'PVA', '1D', '2D','2D1D','2D2D','Pre-stretch', 'thickness']
Y_COLS = ['S0', 'eps10']

df_train

,sID,CNT,MXN,GNP,PVA,1D,2D,2D1D,2D2D,Pre-stretch,thickness,S0,eps10
0,bSC-0001,0.200000,0.610000,0.150000,0.040000,1,0,0,0,0,1200,5.574136,0.670663
1,bSC-0002,0.800000,0.050000,0.050000,0.100000,0,0,0,1,100,1200,9.057971,2.064118
2,bSC-0003,0.400000,0.460000,0.100000,0.040000,0,0,0,1,300,1200,13.386881,3.302038
3,bSC-0004,0.400000,0.560000,0.000000,0.040000,0,0,1,0,100,800,9.049774,2.223482
4,bSC-0005,0.150000,0.670000,0.150000,0.030000,0,0,1,0,100,1200,19.920319,2.285800
...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,bSC-0825,0.316543,0.227505,0.433186,0.022766,1,0,0,0,0,800,13.262599,0.746425
142,bSC-0826,0.279193,0.204776,0.492918,0.023113,0,0,0,1,200,1200,5.938242,5.275677
143,bSC-0828,0.474030,0.333257,0.135878,0.056835,0,0,1,0,100,1600,9.345794,2.601333
144,bSC-0829,0.378706,0.452909,0.127412,0.040973,0,0,1,0,200,1600,11.668611,4.207135


In [3]:
augmente_data(df_train, X_COLS, Y_COLS, n_augmented=1000, x_std=0.05, y_std=0.05, random_state=0)

,sID,DA#,CNT,MXN,GNP,PVA,1D,2D,2D1D,2D2D,Pre-stretch,thickness,S0,eps10
0,bSC-0001,0,0.200000,0.610000,0.150000,0.040000,1.000000,0.000000,0.000000,0.000000,0.000000,1200.000000,5.574136,0.670663
1,bSC-0001,1,0.206287,0.603395,0.182021,0.045245,0.973217,0.018080,0.065200,0.047354,-0.035187,1199.936729,5.444682,0.723737
2,bSC-0001,2,0.222552,0.622811,0.149981,-0.066630,1.078336,0.044921,0.001272,0.022935,-0.046969,1200.025614,5.629337,0.688784
3,bSC-0001,3,0.157413,0.584869,0.164554,0.041052,0.989384,0.058646,-0.017480,-0.006902,0.027698,1200.013539,5.684405,0.600195
4,bSC-0001,4,0.185687,0.655599,0.111245,-0.020272,1.006448,0.024026,-0.041763,0.006277,-0.059791,1200.006315,5.509027,0.669576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146141,bSC-0830,996,0.790499,0.171908,0.023757,0.001174,-0.024251,-0.023310,-0.047725,0.921606,300.085012,1600.004937,7.889851,5.698691
146142,bSC-0830,997,0.671532,0.133457,0.194638,0.091608,-0.059851,-0.037990,-0.051224,0.982504,300.086880,1600.078134,7.789533,5.685013
146143,bSC-0830,998,0.737644,0.068915,0.115452,0.040629,-0.089992,-0.078978,0.051130,1.026074,300.003517,1599.978349,7.850645,5.641682
146144,bSC-0830,999,0.673701,0.116901,0.109678,0.062517,-0.027967,-0.018088,-0.048024,1.020464,300.076425,1600.106773,7.901526,5.653580
